In [42]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import itertools
from zeugma import EmbeddingTransformer

In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
df_train = pd.read_csv('./final_fewshot_train.csv')
df_test = pd.read_csv('./final_fewshot_test.csv')
df_train=df_train[['text','class']]
df_test=df_test[['text','class']]

In [9]:
df_train.head()

,text,class
0,[ALLUXIO-2743] Fix failing unit tests,1
1,#2 Refactored structure of Argument,3
2,Remove some features from JwtTokenStore,4
3,Remove duplicated 1.613 section from changelog,2
4,* webapp structure refactoring,3


In [10]:
df_train.groupby('class').count()

,text
class,
1,20
2,20
3,20
4,20
5,20


We have 20 examples each for 5 classes

In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3277 entries, 0 to 3276
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3277 non-null   object
 1   class   3277 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 51.3+ KB


In [16]:
labels = df_train['class'].unique()
labels

array([1, 3, 4, 2, 5], dtype=int64)

In [38]:
text_left = []
text_right = []
target = []


for label in labels:
    
    similar_texts = df_train[df_train['class']==label]['text']
    group_similar_texts = list(itertools.combinations(similar_texts,2))
    
    text_left.extend([group[0] for group in group_similar_texts])
    text_right.extend([group[1] for group in group_similar_texts])
    target.extend([1.]*len(group_similar_texts))

    dissimilar_texts = df_train[df_train['class']!=label]['text']
    for i in range(len(group_similar_texts)):
        text_left.append(np.random.choice(similar_texts))
        text_right.append(np.random.choice(dissimilar_texts))
        target.append(0.)
        
dataset = pd.DataFrame({'text_left':text_left,
                    'text_right':text_right,
                    'target': target})

In [40]:
dataset.sample(10)

,text_left,text_right,target
1289,Remove duplicate code block,Unify and encapsulate deployment flow methods ...,1.0
1850,CORE-2985 MSSQL Snapshot performance improvements,"Fix for ISSUE 1111 (and 1112, which is a dupli...",0.0
1515,"Refactor test code, remove duplicates at Failo...",#691 - Performance Improvements,0.0
1446,No need to duplicate close method as it's alre...,CORE-2985 MSSQL Snapshot performance improvements,0.0
1791,Improved performance on management of collections,WFLY-6069 remove duplicate jipijapa module in ...,0.0
1588,Improved performance on management of collections,Tiny performance improvement.,1.0
834,"Added tags and lines attributes to @Feature, m...",no need now to get version from parent and bes...,1.0
127,[Automation] - Fix test failure for test_02_re...,Catch all exceptions when change engine and ro...,1.0
1893,Further improve tag testing.,Improve autosave and backup feature (#2194),0.0
549,#935 - Refactor cache settings for structured ...,* Refactoring structure in to modules (mostly ...,1.0


In [41]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   text_left   1900 non-null   object 
 1   text_right  1900 non-null   object 
 2   target      1900 non-null   float64
dtypes: float64(1), object(2)
memory usage: 44.7+ KB


From a training set of 100 samples were able to create 1900 samples for training the siamese network.

## Model

In [45]:
glove  = EmbeddingTransformer('glove')

In [46]:
glove.embedding_dimension

25

In [48]:
embedding_layer = glove.model.get_keras_embedding()

https://data4thought.com/fewshot_learning_nlp.html#Building-the-pairs-dataset